In [11]:
import tensorflow as tf

import io
import os
import unicodedata
import re

### Import dataset

In [2]:
# move current directory to get to access dataset directory
benchmark_directory = os.getcwd()
os.chdir(os.path.join(benchmark_directory, './..'))

In [17]:
def create_dataset(path, num_examples):
    lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

    word_pairs = [[w for w in l.split('\t')]  for l in lines[:num_examples]]

    return zip(*word_pairs)

In [18]:
# start with 30000 for local training
en, jp = create_dataset('./datasets/jesc-corpus.txt', 30000)

In [19]:
en[:10]

("you are back, aren't you, harold?",
 'my opponent is shark.',
 'this is one thing in exchange for another.',
 "yeah, i'm fine.",
 "don't come to the office anymore. don't call me either.",
 'looks beautiful.',
 'get him out of here, because i will fucking kill him.',
 'you killed him!',
 'okay, then who?',
 'it seems a former employee...')

In [20]:
jp[:10]

('あなたは戻ったのね ハロルド?',
 '俺の相手は シャークだ。',
 '引き換えだ ある事とある物の',
 'もういいよ ごちそうさま ううん',
 'もう会社には来ないでくれ 電話もするな',
 'きれいだ。',
 '連れて行け 殺しそうだ わかったか?',
 '殺したのか!',
 'わぁ~! いつも すみません。 いいのよ~。',
 'カンパニーの元社員が')